Delta Task 2 Project WIP

In [1]:
!pip install wordninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541530 sha256=11adefd6b01343e2dcc79470b207c1993dbb575fa3153bb88975acd060fac01c
  Stored in directory: /root/.cache/pip/wheels/e6/66/9c/712044a983337f5d44f90abcd244bd4b8ad28ee64750404b50
Successfully built wordninja


In [56]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import wordninja

In [4]:
#Reading train and test sets into data frame

btc_tweets_train = pd.read_parquet('btc_tweets_train.parquet.gzip')
btc_tweets_test = pd.read_parquet('btc_tweets_test.parquet.gzip')

### Exploratory Analysis to Understand the Data

First start with the exploratory data analysis to understand the data structure. Both training and test data has 5 columns and 1500 observations in train and 500 in test set.

In [26]:
btc_tweets_train.info()
btc_tweets_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, 1641579121972236290 to 1641861677149822976
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   hashtags          1500 non-null   object
 1   content           1500 non-null   object
 2   username          1500 non-null   object
 3   user_displayname  1500 non-null   object
 4   sentiment         1500 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 60.1+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 1641861708246552576 to 1641953216999968769
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   hashtags          500 non-null    object
 1   content           500 non-null    object
 2   username          500 non-null    object
 3   user_displayname  500 non-null    object
 4   sentiment         500 non-null    bool  
dtypes: bool(1), object(4)
memory usage

Content column has the content for sentiment analysis. The tweets include hashtags, emojis and line break elements, which may require handling before we start on tokenizing our data. And it is a mix of uppercase and lower case values

In [32]:
btc_tweets_train['content'][1641861088483368964]

'Last chance below 30k #BITCOIN\u202f\u202f\u202f #LFG \nSet a reminder for my upcoming Space! https://t.co/rnE6XDLOPH'

The data frame has an index column and the indexes for observations are the tweet_ID

In [5]:
btc_tweets_train.head(10)

,hashtags,content,username,user_displayname,sentiment
tweet ID,,,,,
1641579121972236290,"[Bitcoin, Bitcoin, BTC, Bitcoin, BTC, SHIB, HO...","$Bitcoin TO $100,000 SOONER THAN YOU THINK‼️💯🙏...",BezosCrypto,SHIB Bezos,True
1641579176171016194,"[Bitcoin, bitcoinordinals, crypto]",Alright I have my rares. Who else is grabbing ...,spartantc81,SpartanTC,True
1641579486071390208,"[BTC, SHIB, HOGE, SAITAMA, BNB, DOGE, ETH, Bab...","Bitcoin (BTC) Targets Over $100,000 as This Im...",BezosCrypto,SHIB Bezos,True
1641579537103302656,[BTC],📢 Xverse Web-based pool is live:\n\n•Update @x...,godfred_xcuz,Algorithm.btc,True
1641579588399804418,[Bitcoin],"Yesterday, a Bitcoin projection was displayed ...",goddess81oo,she is lucky,True
1641579652681543680,"[Crypto, Bitcoin, Investing]",Unpopular opinion:\n\nThis pump isn’t going to...,CloseSomeSayles,Andrew Sayles☕,False
1641579895615569921,[Bitcoin],#Bitcoin fixes this,ShannenJPEG,Shannen 🍉,True
1641580545678393345,[Bitcoin],Solid bid in major ALT/BTC pairs today. \n\nIf...,tedtalksmacro,tedtalksmacro,True
1641580600606769152,[bitcoin],"If you're filing 2022 taxes in the U.S., you s...",unchainedcom,Unchained,True


The sentiment of the content is already a boolean so dont need to change that. The train set consists heavily of positive sentiment score

In [24]:
btc_tweets_train.describe()

,hashtags,content,username,user_displayname,sentiment
count,1500,1500,1500,1500,1500
unique,945,1500,1012,1012,2
top,[Bitcoin],If the government are allowed to sell all the ...,BezosCrypto,SHIB Bezos,True
freq,384,1,41,41,1220


### Working with a Trial Sample

In [16]:
trial_sample = btc_tweets_train.iloc[5,1]
trial_sample

'Unpopular opinion:\n\nThis pump isn’t going to stop and we are entering a full fledged bull run.\n\nHOWEVER, there will be a major sell off sometime between April and June next year.\n#Crypto #Bitcoin #Investing https://t.co/4fOMtYvY3c'

In [17]:
#Removing of URLs:
preprocessed_tweets_train = re.sub("http\S+"," ",trial_sample)

print(preprocessed_tweets_train)

Unpopular opinion:

This pump isn’t going to stop and we are entering a full fledged bull run.

HOWEVER, there will be a major sell off sometime between April and June next year.
#Crypto #Bitcoin #Investing  


In [18]:
#Changing all tokens to lower case
preprocessed_tweets_train = preprocessed_tweets_train.lower()
print(preprocessed_tweets_train)

unpopular opinion:

this pump isn’t going to stop and we are entering a full fledged bull run.

however, there will be a major sell off sometime between april and june next year.
#crypto #bitcoin #investing  


In [19]:
#Removing the line breaks:
preprocessed_tweets_train = preprocessed_tweets_train.replace('\n'," ")
print(preprocessed_tweets_train)

unpopular opinion:  this pump isn’t going to stop and we are entering a full fledged bull run.  however, there will be a major sell off sometime between april and june next year. #crypto #bitcoin #investing  


In [20]:
#Removing hashtags while replacing the rest of the hastag with the splitted version of the words in hashtag
hashtags = re.findall(r'#\w+', preprocessed_tweets_train)
for tag in hashtags:
  words_in_hashtag = wordninja.split(tag[1:]) #remove '#' and split
  preprocessed_tweets_train = preprocessed_tweets_train.replace(tag, ' '.join(words_in_hashtag))

print(preprocessed_tweets_train)

unpopular opinion:  this pump isn’t going to stop and we are entering a full fledged bull run.  however, there will be a major sell off sometime between april and june next year. crypto bitcoin investing  


In [21]:
#Removing of non-alphabetic characters except numbers:
preprocessed_tweets_train = re.sub(r"[^a-zA-Z0-9,.]", " ", preprocessed_tweets_train) #keeping ',' and '.' to not split numbers like 100,000
print(preprocessed_tweets_train)

unpopular opinion   this pump isn t going to stop and we are entering a full fledged bull run.  however, there will be a major sell off sometime between april and june next year. crypto bitcoin investing  


In [22]:
#Tokenization:
nltk.download('punkt_tab')
preprocessed_tweets_train = nltk.word_tokenize(preprocessed_tweets_train)
print(preprocessed_tweets_train)

['unpopular', 'opinion', 'this', 'pump', 'isn', 't', 'going', 'to', 'stop', 'and', 'we', 'are', 'entering', 'a', 'full', 'fledged', 'bull', 'run', '.', 'however', ',', 'there', 'will', 'be', 'a', 'major', 'sell', 'off', 'sometime', 'between', 'april', 'and', 'june', 'next', 'year', '.', 'crypto', 'bitcoin', 'investing']


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [23]:
#Stopwords removal:
nltk.download('stopwords')
words_list = []
for w in preprocessed_tweets_train:
  if w not in stopwords.words("english"):
    words_list.append(w)

print(words_list)

['unpopular', 'opinion', 'pump', 'going', 'stop', 'entering', 'full', 'fledged', 'bull', 'run', '.', 'however', ',', 'major', 'sell', 'sometime', 'april', 'june', 'next', 'year', '.', 'crypto', 'bitcoin', 'investing']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
lemmatizer = WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
preprocessed_textual_sample = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in words_list]

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
print(preprocessed_textual_sample)

['unpopular', 'opinion', 'pump', 'go', 'stop', 'enter', 'full', 'fledge', 'bull', 'run', '.', 'however', ',', 'major', 'sell', 'sometime', 'april', 'june', 'next', 'year', '.', 'crypto', 'bitcoin', 'invest']


### Solving the Problem Continued

In [7]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [24]:
# Creating a POS Tag (Parts of Speech tagging) to use in lemmatization

def get_wordnet_pos(word:str)->str:
    """Map POS tag to first character for lemmatization

    Returns:
    --------
    pos: str
        The positional tag of speech retrieved from wordnet database.
    """

    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    pos=tag_dict.get(tag,wordnet.NOUN)

    return pos

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
def NLP_preprocessing_pipeline(sample:str)->list:
    '''
    Implements an NLP preprocessing pipeline specific for tweets from twitter.

    Parameters:
    -----------
    textual_sample:str
        The input text that requires preprocessing

    Returns:
    --------
    preprocessed_textual_sample:list
        The textual sample after preprocessing steps have been applied.

    '''

    #Preprocessing steps:

    #Removing of URLs:
    preprocessed_textual_sample = re.sub("http\S+"," ",sample)

    #Changing all tokens to lower case
    preprocessed_textual_sample = preprocessed_textual_sample.lower()

    #Removing the line breaks:
    preprocessed_textual_sample = preprocessed_textual_sample.replace('\n'," ")

    #Removing hashtags while replacing the rest of the hastag with the splitted version of the words in hashtag
    hashtags = re.findall(r'#\w+', preprocessed_textual_sample)
    for tag in hashtags:
      words_in_hashtag = wordninja.split(tag[1:])
      preprocessed_textual_sample = preprocessed_textual_sample.replace(tag, ' '.join(words_in_hashtag))

    #Removing of non-alphabetic characters except numbers:
    preprocessed_textual_sample = re.sub(r"[^a-zA-Z0-9,.]", " ", preprocessed_textual_sample) #keeping ',' and '.' to not split numbers like 100,000

    #Tokenization:
    preprocessed_textual_sample = nltk.word_tokenize(preprocessed_textual_sample)

    #Stopwords removal:
    words_without_stopwords = []

    for w in preprocessed_textual_sample:
      if w not in stopwords.words("english"):
        words_without_stopwords.append(w)

    preprocessed_textual_sample = words_without_stopwords

    # Lemmatize with POS Tag (Parts of Speech tagging)
    lemmatizer = WordNetLemmatizer()
    preprocessed_textual_sample = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in preprocessed_textual_sample]

    return preprocessed_textual_sample


In [33]:
#Creating the preprocessed train and test sets with the preprocessing function
x_tweets_clean_train =  btc_tweets_train['content'].apply(NLP_preprocessing_pipeline)
x_tweets_clean_test =  btc_tweets_test['content'].apply(NLP_preprocessing_pipeline)

#Creating a df for the sentiment labels of train and test sets
y_tweets_train = btc_tweets_train['sentiment']
y_tweets_test = btc_tweets_test['sentiment']

### Dictionary Based Sentiment Analysis

In [57]:
import pysentiment2 as ps
analyzer = ps.HIV4()

def get_sentiment_score(text):

    score = round(analyzer.get_score(text)['Polarity'], 2)
    return score

db_test_scores = x_tweets_clean_test.apply(get_sentiment_score)
db_test_classes = np.where(db_test_scores>0, 1, 0)